Loading & Inspecting Data

In [ ]:
import pandas as pd

print("Loading IPL Auction Dataset...")
df = pd.read_csv("IPLPlayerAuctionData.csv")

print(f"Dataset loaded with {df.shape[0]} rows and {df.shape[1]} columns.\n")

print("Column names:")
print(df.columns)

print("\nPreview of the data:")
print(df.head())

print("\nDataset info:")
print(df.info())


Data Cleaning

In [ ]:
print("Cleaning column names...")
df.columns = df.columns.str.strip().str.replace(' ', '_')

print("Filling missing 'Year' values...")
df['Year'] = df['Year'].ffill().astype(int)

Adding Required Columns

In [ ]:
print("Creating 'Amount_Cr' column...")
df['Amount_Cr'] = df['Amount'] / 1e7

print("Calculating player appearance counts...")
df['Count'] = df.groupby('Player')['Player'].transform('count')

print("Marking repeated players...")
df['Repeated'] = df['Count'] > 1

Saving The Cleaned Data

In [ ]:
print("Saving cleaned data to CSV...")
df.to_csv("Cleaned_IPL_Auction_Data.csv", index=False)
print("File saved successfully.")

Importing Cleaned Data to MySQL. (Optional If you are using SQL database server.)

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Step 1: Load the cleaned CSV file
df = pd.read_csv("Cleaned_IPL_Auction_Data.csv")
print("CSV loaded with shape:", df.shape)

# Step 2: Connect to MySQL Database
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Rishabh@2024',  # replace with your MySQL root password
        database='ipl_auction'
    )

    if connection.is_connected():
        print("Connected to MySQL!")

        cursor = connection.cursor()

        # Step 3: Prepare INSERT query
        insert_query = """
            INSERT INTO auction_data 
            (Player, Role, Amount, Team, Year, Player_Origin, Amount_Cr, Count, Repeated)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Step 4: Insert each row
        for i, row in df.iterrows():
            cursor.execute(insert_query, tuple(row))
            if i % 100 == 0:
                connection.commit()  # commit every 100 rows to avoid memory issues

        connection.commit()
        print(f"All {df.shape[0]} rows inserted successfully.")

except Error as e:
    print("Error while connecting to MySQL:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed.")
